In [23]:
import os

import numpy as np
import pandas as pd

import torch
from torch import nn

In [24]:
import torch.utils.data as data
# 定义dataset
class my_Dataset(data.Dataset):
    def __init__(self, features, labels):
        self.X = features
        self.y = labels

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.X.shape[0]

In [173]:
nyc_data = pd.read_csv('./foursquare-checkin/FS_NYC.csv')
venue_id2index={id:index for index, id in enumerate(nyc_data['venueId'].drop_duplicates())}
nyc_data['venueIndex'] = nyc_data['venueId'].map(venue_id2index)

In [178]:
print(nyc_data['venueIndex'].max())
print(nyc_data['venueIndex'].min())

38332
0


In [181]:
class FourSquareDataset:
    def __init__(self, data_path):
        # 适当修改数据读取过程
        self.nyc_data = pd.read_csv(data_path)
        self.venue_id2index={id:index for index, id in enumerate(self.nyc_data['venueId'].drop_duplicates())}
        self.nyc_data['venueIndex'] = self.nyc_data['venueId'].map(self.venue_id2index)
        self.min = self.nyc_data['venueIndex'].min()
        self.max = self.nyc_data['venueIndex'].max()
        self.nyc_data['venueIndex'] = (self.nyc_data['venueIndex'] - self.min) / (self.max - self.min)

    def denormalize(self, x):
        return x * (self.max - self.min) + self.min

    def construct_set(self, train_por, val_por, test_por, window_size, label=0):
        train_x = []
        train_y = []
        val_x = []
        val_y = []
        test_x = []
        test_y = []

        # 补全构造过程
        for user_id, group in self.nyc_data.groupby('userId'):
            # pandas会对userId进行遍历。
            # 每次遍历中，group包含了对应userId所有的check-in记录。
            user_trajectory = group.sort_values(['utcTimestamp'])['venueIndex'].tolist()

            user_trajectory_length = int(len(user_trajectory))
            train_trajectory_length = int(user_trajectory_length*train_por)
            val_trajectory_length = int(user_trajectory_length*val_por)

            train_trajectory = user_trajectory[:train_trajectory_length]
            val_trajectory = user_trajectory[train_trajectory_length:val_trajectory_length+train_trajectory_length]
            test_trajectory = user_trajectory[train_trajectory_length+val_trajectory_length:]

            for i in range(len(train_trajectory) - window_size):
                train_x.append(train_trajectory[i:i+window_size])
                train_y.append(train_trajectory[i+window_size])
            
            for i in range(len(val_trajectory) - window_size):
                val_x.append(val_trajectory[i:i+window_size])
                val_y.append(val_trajectory[i+window_size])

            for i in range(len(test_trajectory) - window_size):
                test_x.append(test_trajectory[i:i+window_size])
                test_y.append(test_trajectory[i+window_size])            
           
        train_x = torch.tensor(train_x).unsqueeze(2)
        train_y = torch.tensor(train_y).type(torch.LongTensor)
        val_x = torch.tensor(val_x).unsqueeze(2)
        val_y = torch.tensor(val_y).type(torch.LongTensor)
        test_x = torch.tensor(test_x).unsqueeze(2)
        test_y = torch.tensor(test_y).type(torch.LongTensor)
        print(train_x.shape)
        print(train_y.shape)
        print(train_y.type())
        train_set = my_Dataset(train_x, train_y)
        val_set = my_Dataset(val_x, val_y)
        test_set = my_Dataset(test_x, test_y)
        return train_set, val_set, test_set

In [182]:
FourSquareData = FourSquareDataset('./foursquare-checkin/FS_NYC.csv')
train_set, val_set, test_set = FourSquareData.construct_set(train_por=0.6,val_por=0.2,test_por=0.2,window_size=12)
batch_size = 64
train_loader = data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
val_loader = data.DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)
test_loader = data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

torch.Size([123027, 12, 1])
torch.Size([123027])
torch.LongTensor


In [183]:
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        :param input_size: 指定输入数据的维度。例如，对于简单的时间序列预测问题，每一步的输入均为一个采样值，因此input_size=1.
        :param hidden_size: 指定隐藏状态的维度。这个值并不受输入和输出控制，但会影响模型的容量。
        :param output_size: 指定输出数据的维度。此值取决于具体的预测要求。例如，对简单的时间序列预测问题，output_size=1.
        """
        super().__init__()
        self.hidden_size = hidden_size

        # 可学习参数的维度设置，可以类比一下全连接网络的实现。其维度取决于输入数据的维度，以及指定的隐藏状态维度。
        self.w_h = nn.Parameter(torch.rand(input_size, hidden_size))
        self.u_h = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_h = nn.Parameter(torch.zeros(hidden_size))

        self.w_y = nn.Parameter(torch.rand(hidden_size, output_size))
        self.b_y = nn.Parameter(torch.zeros(output_size))

        # 准备激活函数。Dropout函数可选。
        self.tanh = nn.Tanh()
        self.leaky_relu = nn.LeakyReLU()

        # 可选：使用性能更好的参数初始化函数
        for param in self.parameters():
            if param.dim() > 1:
                nn.init.xavier_uniform_(param)

    def forward(self, x):
        """
        :param x: 输入序列。一般来说，此输入包含三个维度：batch，序列长度，以及每条数据的特征。
        """
        #print(x.shape)
        batch_size = x.size(0)
        seq_len = x.size(1)
        # x batch_size X seq_len X input_size
        # 初始化隐藏状态，一般设为全0。由于是内部新建的变量，需要同步设备位置。
        h = torch.zeros(batch_size, self.hidden_size).to(x.device)
        # RNN实际上只能一步一步处理序列。因此需要用循环迭代。
        y_list = []
        for i in range(seq_len):
            h = self.tanh(torch.matmul(
                x[:, i, :], self.w_h) + torch.matmul(h, self.u_h) + self.b_h)
            # (batch_size, output_size)
            y = self.leaky_relu(torch.matmul(h, self.w_y) + self.b_y)
            y_list.append(y)  # seq_len X batch_size X output_size
        # 一般来说，RNN的返回值为最后一步的隐藏状态，以及每一步的输出状态。
        out_y = torch.stack(y_list, dim=1)
        y_hat = out_y[:, -1, :]
        #print(out_y.shape)
        #print(y_hat.shape)
        #print(out_y.shape)
        #print(h.shape)
        return out_y, h


In [184]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = train_set.X.shape[-1]
hidden_size = 64

output_size = 38333

print(input_size, hidden_size, output_size)
seq_len = 12
lr = 0.0001
epochs = 80
loss_func = nn.CrossEntropyLoss()

my_rnn = MyRNN(input_size, hidden_size, output_size).to(device)

optimizer = torch.optim.Adam(my_rnn.parameters(), lr)

1 64 38333


In [169]:
def evaluate_accuracy(data_iter,net,loss):
    acc_sum,n=0.0,0
    test_l_sum=0.0
    for X,y in data_iter:
        X = X.to(device)
        y = y.to(device)        
        acc_sum+=(net(X).argmax(dim=1)==y).float().sum().item()
        l=loss(net(X),y).sum()
        test_l_sum+=l.item()
        n+=y.shape[0]
    return acc_sum/n,test_l_sum/n

In [170]:
def train(net,train_iter,test_iter,loss,num_epochs,batch_size,params=None,lr=None,optimizer=None):
    train_loss=[]
    test_loss=[]
    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n=0.0,0.0,0
        for X,y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat,h = net(X)
            y_hat=y_hat[:,0,:]
            print(y.shape)#64
            print(y_hat.shape)#64X1
            print(y.type())#expect float got long???
            print(y_hat.type())#float
            y=torch.unsqueeze(y,1)#target 二维的 64X1
            print(y.shape)
            l=loss(y_hat,y).sum()
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            if optimizer is None:
                SGD(params,lr)
            else:
                optimizer.step()
            train_l_sum+=l.item()
            train_acc_sum+=(y_hat.argmax(dim=1)==y).sum().item()
            n+=y.shape[0]
        test_acc,test_l = evaluate_accuracy(test_iter,net,loss)
        train_loss.append(train_l_sum/n)
        test_loss.append(test_l)
        print('epoch%d,loss%.4f,train acc %3f,test acc %.3f'%(epoch+1,train_l_sum/n,train_acc_sum/n,test_acc))
    return train_loss,test_loss

In [185]:
train_loss,test_loss=train(my_rnn,train_loader,test_loader,loss_func,epochs,batch_size,my_rnn.parameters(),lr,optimizer)

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
import math
import torch
from torch.utils import data
import torch.nn as nn
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error as mse_fn, mean_absolute_error as mae_fn
import numpy as np
import time


def mape_fn(y, pred):
    mask = y != 0
    y = y[mask]
    pred = pred[mask]
    mape = np.abs((y - pred) / y)
    mape = np.mean(mape) * 100
    return mape


def eval(y, pred):
    y = y.cpu().numpy()
    pred = pred.cpu().numpy()
    mse = mse_fn(y, pred)
    rmse = math.sqrt(mse)
    mae = mae_fn(y, pred)
    mape = mape_fn(y, pred)
    return [rmse, mae, mape]


# 测试函数（用于分类）
def test(net, output_model, data_iter, loss_fn, denormalize_fn, device='cpu'):
    rmse, mae, mape = 0, 0, 0
    batch_count = 0
    total_loss = 0.0
    net.eval()
    if output_model is not None:
        output_model.eval()
    for X, Y in data_iter:
        X = X.to(device).float()
        Y = Y.to(device).float()
        output, hidden = net(X)
        if output_model is not None:
            y_hat = output_model(output[:, -1, :].squeeze(-1)).squeeze(-1)
        else:
            y_hat = output[:, -1, :].squeeze(-1)
        loss = loss_fn(y_hat, Y)

        Y = denormalize_fn(Y)
        y_hat = denormalize_fn(y_hat)
        a, b, c = eval(Y.detach(), y_hat.detach())
        rmse += a
        mae += b
        mape += c
        total_loss += loss.detach().cpu().numpy().tolist()
        batch_count += 1
    return [rmse / batch_count, mae / batch_count, mape / batch_count], total_loss / batch_count


def train(net, train_iter, val_iter, test_iter, loss_fn, denormalize_fn, optimizer, num_epoch,
          early_stop=10, device='cpu', output_model=None, is_print=True, is_print_batch=False):
    train_loss_lst = []
    val_loss_lst = []
    train_score_lst = []
    val_score_lst = []
    epoch_time = []

    best_epoch = 0
    best_val_rmse = 9999
    early_stop_flag = 0
    for epoch in range(num_epoch):
        net.train()
        if output_model is not None:
            output_model.train()
        epoch_loss = 0
        batch_count = 0
        batch_time = []
        rmse, mae, mape = 0, 0, 0
        for X, Y in train_iter:
            batch_s = time.time()
            X = X.to(device).float()
            Y = Y.to(device).float()
            optimizer.zero_grad()
            output, hidden = net(X)
            if output_model is not None:
                y_hat = output_model(output[:, -1, :].squeeze(-1)).squeeze()
            else:
                y_hat = output[:, -1, :].squeeze(-1)
            loss = loss_fn(y_hat, Y)#input,target
            loss.backward()
            optimizer.step()

            Y = denormalize_fn(Y)
            y_hat = denormalize_fn(y_hat)
            a, b, c = eval(Y.detach(), y_hat.detach())
            rmse += a
            mae += b
            mape += c
            epoch_loss += loss.detach().cpu().numpy().tolist()
            batch_count += 1
            # sample_num += X.shape[0]

            batch_time.append(time.time() - batch_s)
            if is_print and is_print_batch:
                print('epoch-batch: %d-%d, train loss %.4f, time use %.3fs' %
                      (epoch + 1, batch_count, epoch_loss, batch_time[-1]))

        train_loss = epoch_loss / batch_count
        train_loss_lst.append(train_loss)
        train_score_lst.append([rmse/batch_count, mae/batch_count, mape/batch_count])

        # 验证集
        val_score, val_loss = test(net, output_model, val_iter, loss_fn, denormalize_fn, device)
        val_score_lst.append(val_score)
        val_loss_lst.append(val_loss)

        epoch_time.append(np.array(batch_time).sum())

        # 打印本轮训练结果
        if is_print:
            print('*** epoch%d, train loss %.4f, train rmse %.4f, val loss %.4f, val rmse %.6f, time use %.3fs' %
                  (epoch + 1, train_loss, train_score_lst[-1][0], val_loss, val_score[0], epoch_time[-1]))

        # 早停
        if val_score[0] < best_val_rmse:
            best_val_rmse = val_score[0]
            best_epoch = epoch
            early_stop_flag = 0
        else:
            early_stop_flag += 1
            if early_stop_flag == early_stop:
                print(f'\nThe model has not been improved for {early_stop} rounds. Stop early!')
                break

    # 输出最终训练结果
    print(f'\n{"*" * 40}\nFinal result:')
    print(f'Get best validation rmse {np.array(val_score_lst)[:, 0].min() :.4f} '
          f'at epoch {best_epoch}')
    print(f'Total time {np.array(epoch_time).sum():.2f}s')
    print()

    # 计算测试集效果
    test_score, test_loss = test(net, output_model, test_iter, loss_fn, denormalize_fn, device)
    print('Test result:')
    print(f'Test RMSE: {test_score[0]}    Test MAE: {test_score[1]}    Test MAPE: {test_score[2]}')
    return train_loss_lst, val_loss_lst, train_score_lst, val_score_lst, epoch


def visualize(num_epochs, train_data, test_data, x_label='epoch', y_label='loss'):
    x = np.arange(0, num_epochs + 1).astype(dtype=np.int)
    plt.plot(x, train_data, label=f"train_{y_label}", linewidth=1.5)
    plt.plot(x, test_data, label=f"val_{y_label}", linewidth=1.5)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.show()


def plot_metric(score_log):
    score_log = np.array(score_log)

    plt.figure(figsize=(10, 6), dpi=300)
    plt.subplot(2, 2, 1)
    plt.plot(score_log[:, 0], c='#d28ad4')
    plt.ylabel('RMSE')

    plt.subplot(2, 2, 2)
    plt.plot(score_log[:, 1], c='#e765eb')
    plt.ylabel('MAE')

    plt.subplot(2, 2, 3)
    plt.plot(score_log[:, 2], c='#6b016d')
    plt.ylabel('MAPE')

    plt.show()

In [186]:
from utils import *

train_loss_lst, val_loss_lst, \
    train_score_lst, val_score_lst, stop_epoch = train(my_rnn, train_loader, val_loader, test_loader,
                                                       loss_func, FourSquareData.denormalize, optimizer, epochs,
                                                       early_stop=20, device=device, output_model=None)

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

In [ ]:
visualize(stop_epoch, train_loss_lst, val_loss_lst, y_label='Loss')
plot_metric(train_score_lst)